In [16]:
import lectura_paco_colusiones
import lectura_paco_multas
import lectura_paco_secop
import lectura_secop
import pyspark_etl_functions

In [10]:
url = 'https://paco7public7info7prod.blob.core.windows.net/paco-pulic-info/colusiones_en_contratacion_SIC.csv'
filename = 'colusiones_en_contratacion_SIC.csv'
lectura_paco_colusiones.download_and_save_csv(url, filename)

Archivo 'colusiones_en_contratacion_SIC.csv' guardado en la carpeta 'data'.


In [12]:
url = 'https://paco7public7info7prod.blob.core.windows.net/paco-pulic-info/multas_SECOP.csv'
filename = 'multas_SECOP.csv'
lectura_paco_multas.download_and_save_csv(url, filename)

Archivo 'multas_SECOP.csv' guardado en la carpeta 'data'.


In [13]:
url = 'https://paco7public7info7prod.blob.core.windows.net/paco-pulic-info/SECOP_II_Cleaned.csv'
filename = 'SECOP_II_Cleaned.csv'
lectura_paco_secop.download_and_save_csv(url, filename)

Archivo 'SECOP_II_Cleaned.csv' guardado en la carpeta 'data'.


In [14]:
lectura_secop.main()

Se han actualizado los datos en el archivo CSV.


In [17]:
filename = 'colusiones_en_contratacion_SIC.csv'
colusiones_df = pyspark_etl_functions.read_csv_with_pyspark(filename)
colusiones_df.show(5)

23/04/11 16:53:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
+---+-------------------+--------+-----------+----------------------------+----------------------+---------------------+--------------------------+--------------------+--------------+-------------+--------------+
|No.|Fecha de Radicacion|Radicado|       Caso|Falta que origina la sancion|Resolucion de Apertura|Resolucion de Sancion|Tipo de Persona Sancionada|Personas Sancionadas|Identificacion|Multa Inicial|Año Radicacion|
+---+-------------------+--------+-----------+----------------------------+----------------------+---------------------+--------------------------+--------------------+--------------+-------------+--------------+
|  1|2011-06-07 00:00:00|11-71590|VIGILANCIA |        Infringir el artí...|  Resolución de Ape...| Resolución de San...|        Personas Jurídicas|GUARDIANES COMPAÑ...|     860520097| 8.59440305E9|          2011|
|  1|2011-06-07 00:00:00|11-

In [19]:
pyspark_etl_functions.analyze_data_quality('multas_SECOP.csv', folder='data')

Total de registros: 1444

Conteo de registros nulos por columna:
nombre_entidad: 0
nit_entidad: 0
nivel: 0
orden: 0
numero_de_resolucion: 0
documento_contratista: 0
nombre_contratista: 0
numero_de_contrato: 0
valor_sancion: 0
fecha_de_publicacion: 0
ruta_de_proceso: 0
cod_depto: 11
cod_mpio: 11
dpto: 0
nom_mpio: 0

Conteo de registros duplicados: 0

Estadísticas descriptivas para columnas numéricas:
min:
  documento_contratista: 230802
  valor_sancion: 0.0
  cod_depto: 5
  cod_mpio: 5001
max:
  documento_contratista: 90075612726
  valor_sancion: 9.7969201936E10
  cod_depto: 99
  cod_mpio: 99001
mean:
  documento_contratista: 4.0192013053026314E9
  valor_sancion: 6.246716858476892E8
  cod_depto: 29.314026517794836
  cod_mpio: 29432.270062805303
stddev:
  documento_contratista: 6.43643826806861E9
  valor_sancion: 6.293069421343648E9
  cod_depto: 26.57250921265884
  cod_mpio: 26605.777653086523

Valores atípicos en la columna documento_contratista: 5

Valores atípicos en la columna valor_